In [1]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [175]:
df = pd.read_csv('data/task_environments_2021-07-15_transformed.csv', header=0, index_col=0)

In [3]:
### Select columns by type

In [176]:
float_cols = df.select_dtypes(include=['float64']).columns
str_cols = df.select_dtypes(include=['object']).columns

In [21]:
### Fill NAs

In [177]:
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)
#df.loc[:, str_cols] = df.loc[:, str_cols].fillna('u')

In [23]:
### Convert str to bool for true/false values

In [178]:
mask = df.applymap(type) != bool
bool_to_str = {True: 1, False: 0}
df = df.where(mask, df.replace(bool_to_str))

In [196]:
df

,sequence,seq1,seq2,seq3,seq4,seq5,seq6,seq7,seq8,seq9,...,coordinates_l.z,coordinates_a.x,coordinates_a.y,coordinates_a.z,coordinates_h.x,coordinates_h.y,coordinates_h.z,coordinates_q.x,coordinates_q.y,coordinates_q.z
0,pocgkr,p,o,c,g,k,r,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,cgwpcfks,c,g,w,p,c,f,k,s,NaN,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,kfsfkspwg,k,f,s,f,k,s,p,w,g,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,pfkswkfsococg,p,f,k,s,w,k,f,s,o,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,wptgkfsoc,w,p,t,g,k,f,s,o,c,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,hhsdbg,h,h,s,d,b,g,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,0.0,4.0,1.0,-99.0,-99.0,-99.0
186,hhsdgb,h,h,s,d,g,b,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,0.0,4.0,1.0,-99.0,-99.0,-99.0
187,hhsgdb,h,h,s,g,d,b,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,0.0,4.0,1.0,-99.0,-99.0,-99.0
188,hhsgdb,h,h,s,g,d,b,NaN,NaN,NaN,...,-99.0,-99.0,-99.0,-99.0,0.0,4.0,1.0,-99.0,-99.0,-99.0


In [180]:
# get unique values in sequences
sequence_list = [list(row) for row in df['sequence']]
unique_items = set([item for sublist in sequence_list for item in sublist])

len(unique_items)

21

In [255]:
list_dicts = []
input_target_values = []
list_already_seen = [{item: 0 for item in unique_items} for x in range(0,1491)]

for row in range(0, len(df)):
    row_dict = {}
    already_seen = []
    
    for position, char in enumerate(df.loc[row, 'sequence']):
        #print(position +1)
        char_dict = {}
        new_row_nr = row + position
        
        # needs to go into char_dict
        start_coords_col = 'start_coords' + str(position+1)
        row_dict['start_coords.x'] = df.loc[row, str(start_coords_col + '.x')]
        row_dict['start_coords.y'] = df.loc[row, str(start_coords_col + '.y')]
        row_dict['start_coords.z'] = df.loc[row, str(start_coords_col + '.z')]
        
        # can go into row_dict
        char_coordinates_x = 'coordinates_' + char + '.x'
        char_coordinates_y = 'coordinates_' + char + '.y'
        char_coordinates_z = 'coordinates_' + char + '.z'
        row_dict[char_coordinates_x] = df.loc[row, char_coordinates_x]
        row_dict[char_coordinates_y] = df.loc[row, char_coordinates_y]
        row_dict[char_coordinates_z] = df.loc[row, char_coordinates_z]
            
        char_containment = char + '.containment'
        char_strong_k = char + '.strong_k'
        char_mid_k = char + '.mid_k'
        char_food_k = char + '.food_k'
        row_dict[char_containment] = df.loc[row, char_containment]
        row_dict[char_food_k] = df.loc[row, char_food_k]
        row_dict[char_strong_k] = df.loc[row, char_strong_k]
        row_dict[char_mid_k] = df.loc[row, char_mid_k]
        
        if position == 0:
            input_value = '<start>'
            target_value = char
            input_target_values.append([input_value, target_value])
            
        else:
            input_value = df.loc[row, 'sequence'][position-1]
            target_value = char
            input_target_values.append([input_value, target_value])
            #list_already_seen[new_row_nr][input_value] = 1
            already_seen.append(input_value)
            
        #print(already_seen)
        for item in unique_items:
            if item in already_seen:
                list_already_seen[new_row_nr][item] = 1

                
    list_dicts.append(row_dict)

In [257]:
list_already_seen

[{'o': 0,
  'l': 0,
  'd': 0,
  's': 0,
  'h': 0,
  'c': 0,
  'r': 0,
  'x': 0,
  'q': 0,
  'g': 0,
  'k': 0,
  'a': 0,
  'm': 0,
  'w': 0,
  'f': 0,
  'n': 0,
  'z': 0,
  'e': 0,
  'b': 0,
  'p': 0,
  't': 0},
 {'o': 0,
  'l': 0,
  'd': 0,
  's': 0,
  'h': 0,
  'c': 0,
  'r': 0,
  'x': 0,
  'q': 0,
  'g': 0,
  'k': 0,
  'a': 0,
  'm': 0,
  'w': 0,
  'f': 0,
  'n': 0,
  'z': 0,
  'e': 0,
  'b': 0,
  'p': 1,
  't': 0},
 {'o': 1,
  'l': 0,
  'd': 0,
  's': 0,
  'h': 0,
  'c': 1,
  'r': 0,
  'x': 0,
  'q': 0,
  'g': 0,
  'k': 0,
  'a': 0,
  'm': 0,
  'w': 0,
  'f': 0,
  'n': 0,
  'z': 0,
  'e': 0,
  'b': 0,
  'p': 1,
  't': 0},
 {'o': 1,
  'l': 0,
  'd': 0,
  's': 0,
  'h': 0,
  'c': 1,
  'r': 0,
  'x': 0,
  'q': 0,
  'g': 1,
  'k': 1,
  'a': 0,
  'm': 0,
  'w': 0,
  'f': 0,
  'n': 0,
  'z': 0,
  'e': 0,
  'b': 0,
  'p': 1,
  't': 0},
 {'o': 1,
  'l': 0,
  'd': 0,
  's': 0,
  'h': 0,
  'c': 1,
  'r': 0,
  'x': 0,
  'q': 0,
  'g': 1,
  'k': 1,
  'a': 0,
  'm': 0,
  'w': 1,
  'f': 1,
  'n':

In [258]:
single_step_df = pd.DataFrame(list_dicts)
single_step_df.insert(loc=0, column='input', value=0)
single_step_df.insert(loc=1, column='target', value=0)

In [259]:
for elem in range(0, len(input_target_values)):
    single_step_df.loc[elem, 'input'] = input_target_values[elem][0]
    single_step_df.loc[elem, 'target'] = input_target_values[elem][1]

In [260]:
single_step_df

,input,target,start_coords.x,start_coords.y,start_coords.z,coordinates_p.x,coordinates_p.y,coordinates_p.z,p.containment,p.food_k,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513,-0.531,0.74,0.008041,0.929605,0.689020,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513,-0.531,0.74,0.008041,0.929605,0.689020,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,h,h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,h,s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,s,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,d,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
date = datetime.today().strftime('%Y-%m-%d')
filename = 'data/single_step_df_ints_' + str(date) + '.csv'

filename

'data/single_step_df_ints_2022-03-14.csv'

In [92]:
single_step_df.to_csv(filename, index=False, header=True)